<a href="https://colab.research.google.com/github/rounak393/autopace/blob/main/autopace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics opencv-python

import cv2
import torch
import shutil
from ultralytics import YOLO
from google.colab import files
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device}")

print("Upload your traffic video file (.mp4, .avi, etc.)")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f"Uploaded: {video_path}")

model = YOLO("yolov8n.pt").to(device)
cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

resize_width, resize_height = 1280, 720
if frame_width > 1280 or frame_height > 720:
    print("Resizing to 720p for better speed...")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "fast_processed_video.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (resize_width, resize_height))

start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (resize_width, resize_height))
    results = model(frame, verbose=False)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())

            if cls == 2 and conf > 0.4:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Car {conf:.2f}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()

print(f"Processing done in {time.time() - start_time:.2f} seconds")

shutil.move(output_path, "/content/fast_processed_video.mp4")

print("Download your processed video:")
files.download("/content/fast_processed_video.mp4")

Using cpu
Upload your traffic video file (.mp4, .avi, etc.)


Saving video.mp4 to video (1).mp4
Uploaded: video (1).mp4
Processing done in 881.65 seconds
Download your processed video:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>